In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression 
plt.style.use("bmh")

In [ ]:
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [ ]:
columns=['ori','dst','dist_km','fsize_m6_wd_pm','trip_m6_wd_pm',
                                'community_m6','mrt_km_o','entro_o','cycle_km_o',
                                'far_hdb_o','far_priv_o','far_comm_o','mrt_km_d',
                                'entro_d','cycle_km_d','far_hdb_d','far_priv_d','far_comm_d',
                                'ori_lat','ori_lon','dst_lat','dst_lon']
ycol='trip_m6_wd_pm'
columns.remove(ycol)
features=columns                                                         

In [ ]:
og_data=pd.read_csv('apply_lonal.csv')
y=og_data[ycol]
X=og_data[features]

In [ ]:
scores=make_mi_scores(X, y)

In [ ]:
plot_mi_scores(scores)

In [ ]:
scores

## 这说明地域性很重要，而距离更重要

GeoHash算法：将二维的点数据转换成一维的数据
1、Geohash的最简单的解释就是：将一个经纬度信息，转换成一个可以排序，可以比较的字符串编码。
 
2、geohash有以下几个特点：
首先，geohash用一个字符串表示经度和纬度两个坐标。某些情况下无法在两列上同时应用索引 （例如MySQL 4之前的版本，Google App Engine的数据层等），利用geohash，只需在一列上应用索引即可。
其次，geohash表示的并不是一个点，而是一个矩形区域。比如编码wx4g0ec19，它表示的是一个矩形区域。 使用者可以发布地址编码，既能表明自己位于北海公园附近，又不至于暴露自己的精确坐标，有助于隐私保护。
第三，编码的前缀可以表示更大的区域。例如wx4g0ec1，它的前缀wx4g0e表示包含编码wx4g0ec1在内的更大范围。 这个特性可以用于附近地点搜索。首先根据用户当前坐标计算geohash（例如wx4g0ec1）然后取其前缀进行查询 （SELECT * FROM place WHERE geohash LIKE 'wx4g0e%'），即可查询附近的所有地点。

Geohash比直接用经纬度的高效很多。
 
3、编码方式：
  
首先将纬度范围(-90, 90)平分成两个区间(-90,0)、(0, 90)，如果目标纬度位于前一个区间，则编码为0，否则编码为1。由于39.92324属于(0, 90)，所以取编码为1。
 
然后再将(0, 90)分成 (0, 45), (45, 90)两个区间，而39.92324位于(0, 45)，所以编码为0。

以此类推，直到精度符合要求为止，得到纬度编码为1011 1000 1100 0111 1001。 
经度也用同样的算法，对(-180, 180)依次细分，得到116.3906的编码为1101 0010 1100 0100 0100。